In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import matplotlib
from matplotlib.animation import FFMpegWriter
import string
from copy import deepcopy
import warnings 
from tqdm import tqdm
warnings.filterwarnings('ignore')
###################################################################################################
#############主程序

df = pd.read_csv('./trainData/originFranceData1.csv',sep = ';')
print("#################################################################")
print("df.head()\n",df.head())
#print("df.info()\n",df.info())
#print(df.describe())


print("#################################################################")
print("veh_lane.unique()\n",df.vehicle_lane.unique())
laneList = df.vehicle_lane.unique()
numLanes = len(df.vehicle_lane.unique())
print("numLanes \n",numLanes)
print("#################################################################")


########################################################################################
#######枚举每一个车道,获得红灯附近的车，以及车道的长度
for ilane,curLaneID in enumerate(df.vehicle_lane.unique()):#枚举每一个车道
#for ilane in tqdm(range(numLanes)):#枚举每一个车道
    curLaneID= laneList[ilane]
    
    #提取红灯时刻的车辆样本 LaneID:239331354_0 ; Time:4224.0 ; redID:VehicleFlowSouthToWest4.17
    #if curLaneID != '239331354_0': #for debug
    #   continue

    #1.获得车道长度
    redVehs = pd.DataFrame(columns=df.columns)  #建立空的二维数组，，并且列与数据库一致。
    print("建立空的红灯静止车辆二维数组 redVehs\n")
    
    vehInOneLane = df[df.vehicle_lane==curLaneID]#获得当前车道上所有车辆
    if vehInOneLane.empty == True:
        print("当前车道counter,车道总数,车道名:",ilane,numLanes,curLaneID)
        print("\n vehInOneLane.empty == True,当前车道没有车\n")
        continue
    maxLanePos =  max(vehInOneLane.vehicle_pos)#车道的长度
    print("当前车道counter,车道总数,车道名,车道长度：",ilane,numLanes,curLaneID,maxLanePos)
    ############################################################3
    #2.提取红灯车辆以及时刻
    vehInOneLane =vehInOneLane.sort_values(by='timestep_time',ascending=True)
   
    timeList = vehInOneLane.timestep_time.unique()
    lowSpeedFlag = 0
    for t in timeList:#枚举每个时间
        title = "2.提取红灯车辆以及时刻,LaneID:"+str(curLaneID)+";"+"Time:"+str(t)+";"
        #print(title)
        vehsAtTime = vehInOneLane[vehInOneLane.timestep_time == float(t)] 

        for index, veh in vehsAtTime.iterrows():#每一辆车
            #print(index,veh)
            vehX =  veh.vehicle_x
            vehY =  veh.vehicle_y
            vehVel = veh.vehicle_speed
            vehTime  = veh.timestep_time
            vehID = veh.vehicle_id
            vehicle_pos = veh.vehicle_pos
            if (vehVel <2/3.6) and (abs(vehicle_pos - maxLanePos)<5):#红色静止车辨别标准
                lowSpeedFlag = 1
                redVehs.loc[len(redVehs.index)] =veh
               
   
    
          

    if redVehs.empty == True:
        print("redVehs.empty == True")
        #input()
        continue
    print(redVehs.info())
    print(redVehs.head())

    #3.当有车处于红灯状态时而且车道上有多个车时,对每条道路进行分析
    speedFlagDict = {}
    for ired, redID in enumerate(redVehs.vehicle_id.unique()):
        
        print("3. 分析车道为%s,红灯静止车辆为%s时的情况" %(curLaneID,redID))
        redVehFocusTmp = redVehs[redVehs.vehicle_id == redID]#红灯状态的车辆ID
        timeList = redVehFocusTmp.timestep_time.values  
        # 我们认为红灯静止车在本车道的存在时间，就是红灯剩余时间，也就是
        #max(timeList)-min(timeList)
        
        
        #3.1 枚举当前道路上红灯状态下的时间内所有车，并获得最小速度。、
        #我们认为红灯静止在本车道的存在时间，就是红灯剩余时间
        locTmp1 = vehInOneLane.timestep_time >= min(timeList)
        locTmp2 = vehInOneLane.timestep_time <= (max(timeList)+20)#红灯后灯后的20秒也计算，最小绿灯时间15秒
        #locTmp3 = abs(maxLanePos - vehInOneLane.vehicle_pos)<100 ##距离红灯100米以内,红灯时间内车道内所有的车
        
        vehsAtTimeAndDist = vehInOneLane[locTmp1 & locTmp2]#距离红灯100米以内,红灯时间内车道内所有的车
        vehIDsAtTimeAndDist = vehsAtTimeAndDist.vehicle_id.unique()
        print("3.1 枚举当前道路上红灯状态下的时间内所有车，并获得最小速度\n",vehIDsAtTimeAndDist)
        for ii,idTmp  in enumerate(vehIDsAtTimeAndDist):
            vehTmp = vehsAtTimeAndDist[vehsAtTimeAndDist.vehicle_id== idTmp]#提取距离红灯100米以内,红灯时间内一部车的信息
            minSpeed = min(vehTmp.vehicle_speed.values)#距离红灯100米以内,红灯时间内一部车的是所有速度信息
            if minSpeed >= 35/3.6:
                speedFlag  = 4
            if minSpeed <=35/3.6 and minSpeed> 25/3.6:
                speedFlag  = 3
            if minSpeed <=25/3.6 and minSpeed> 15/3.6:
                speedFlag  = 2
            if minSpeed <=15/3.6 and minSpeed> 5/3.6:
                speedFlag  = 1
            if minSpeed <=5/3.6:
                speedFlag  = 0
            #注意时间分区
            speedFlagDict[idTmp] = speedFlag

        print("3.2 枚举当前道路上当前红灯静止车，红灯状态下的每个时间的每一辆车，并生成样本\n")
        
        samples3 = []#收集当前车道内对应红车有样本
        
        for t in timeList:#枚举红灯状态下的每个时间的每一辆车 
            
            
            locTmp1 =  vehInOneLane.timestep_time == t      
            locTmp2 = abs(maxLanePos - vehInOneLane.vehicle_pos)<100 #距离红灯100米以内

            vehsAtTime = vehInOneLane[locTmp1 & locTmp2] #符合车道，时间和距离限制
            
            if len(vehsAtTime.vehicle_id.unique()) == 1:  # 如果当前时间车辆只有一部车，统计忽略
                #print("vehsAtTime.vehicle_id.unique()) == 1,时间%d,redID:%s\n" %(t,redID))
                #print(vehsAtTime.vehicle_id.unique())
                #print("当前红色静止车的时间范围内的样本不要了\n")
                samples3 = []#当前红色静止车的时间范围内的样本不要了
                continue


            vehsAtTime =vehsAtTime.sort_values(by='vehicle_pos',ascending=False)
            samples = []#当前道路上红灯状态下当前红灯静止车的每个时间的每一辆车，并生成每个时刻样本
            counter = 0
            
            #3.2.1 枚举当前道路上红灯状态下的每个时间的每一辆车，并生成每个时刻样本
            title = "LaneID:"+str(curLaneID)+" ; "+"Time:"+str(t)+" ; "+"redID:"+redID
            print("3.2 举当前道路上红灯状态下当前红灯静止车的当前时间的每一辆车，并生成每个时刻样本\n",title)
            for rowindex, veh in vehsAtTime.iterrows():#每一辆车                  
                vehX =  veh.vehicle_x
                vehY =  veh.vehicle_y
                vehVel = veh.vehicle_speed
                vehTime  = veh.timestep_time
                vehID = veh.vehicle_id
                vehicle_Red_distane = maxLanePos - veh.vehicle_pos
                #print("vehID:",vehID," vehicle_Red_distane:",vehicle_Red_distane)
                samples2 = []
                if counter == 0 and vehID != redID:
                    print("第一部车不是红灯静止车，有错误，counter == 0,redID:",redID,"vehID",vehID)
                    samples3 = []#收集当前车道内对应红车有样本
                    break
                
                    

                if counter > 0:#生成用于机器学习的样本
                    avg_speed_lane = 60/3.6
                    max_speed_lane = 80/3.6
                    subject = [vehID,max(timeList) - t,#主车名，红灯剩余时间
                    vehicle_Red_distane,
                    vehVel,
                    avg_speed_lane,
                    vehicle_Red_distane/(vehVel+0.01),
                    vehicle_Red_distane/avg_speed_lane]
                    
                    samplesTmp = deepcopy(samples)
                    samplesTmp.extend([0, 0]*(20-counter))#主车前面的车（最大20车）车辆的状态

                    samples2 = deepcopy(subject)#当前样本：主车
                    samples2.extend(samplesTmp)#当前样本：主车+主车前面的车（最大20车）
                    samples2.extend([speedFlagDict[vehID]])#当前样本：主车+主车前面的车（最大20车）+speedFlag
                    samples3.append(samples2)#收集当前时间段内所有的样本
                    
                    #print("subject:",subject)
                    #print("samples:",samples)
                    #print("samplesTmp:",samplesTmp)
                    #print("samples2:",samples2)
                    
                        
                #当前道路上红灯状态下当前红灯静止车的每个时间的每一辆车，收集生成每个时刻样本,非机器学习的标准自定义样本     
                samples.extend([vehicle_Red_distane, vehVel])
                counter = counter+1
               
            

               
       



            #记录当前时刻排队情况
            #filename = '.\\franceRedData\\Platoon'+str(ilane)+redID+'+Time@'+str(t)+'.csv'
            #print(filename)
            #print("vehsAtTime.head()\n",vehsAtTime.head())
            #vehsAtTime.to_csv(filename)
        
        
        #当前车道，每个红灯车的所有时刻的样本
        name1 = ["vehID","redLightTime","distToRedLight","speed","laneAvgSpeed","arriveTime1","arriveTime2"]   
        name2 = ["vehPos_1","vehSpeed_1","vehPos_2","vehSpeed_2","vehPos_3","vehSpeed_3","vehPos_4","vehSpeed_4"] 
        name3 = ["vehPos_5","vehSpeed_5","vehPos_6","vehSpeed_6","vehPos_7","vehSpeed_7","vehPos_8","vehSpeed_8"]
        name4 = ["vehPos_9","vehSpeed_9","vehPos_10","vehSpeed_10","vehPos_11","vehSpeed_11","vehPos_12","vehSpeed_12"]
        name5 = ["vehPos_13","vehSpeed_13","vehPos_14","vehSpeed_14","vehPos_15","vehSpeed_15","vehPos_16","vehSpeed_16"]
        name6 = ["vehPos_17","vehSpeed_17","vehPos_18","vehSpeed_18","vehPos_19","vehSpeed_19","vehPos_20","vehSpeed_20"]
        headers = name1+name2+name3+name4+name5+name6+["speedFlag"]

        if samples3 != []:
            samplesTmp = pd.DataFrame(samples3,columns=headers)
            filename = './franceRedData/'+str(ilane)+'+'+redID+'.csv'
            samplesTmp.to_csv(filename,float_format='%.3f',index=0) 


###################################################################################
#将所有样本集合成一个CSV文件
import os
import pandas as pd
path = "./franceRedData/"

filelist = [path + i for i in os.listdir(path)]
dataset = pd.read_csv(filelist[0])

for tmpFile in filelist:
    if tmpFile.endswith(".csv"):
        #print(tmpFile)
        tmpDF = pd.read_csv(tmpFile)
        dataset = pd.concat([dataset,tmpDF],ignore_index=True,axis=0)
        

filename= "./trainData/"+"0_allSamples.csv" 
dataset.to_csv(filename,float_format='%.3f',index=0) 
        



In [9]:
########################重新优化

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import matplotlib
from matplotlib.animation import FFMpegWriter
import string
from copy import deepcopy
import warnings 
from tqdm import tqdm
warnings.filterwarnings('ignore')

###################################################################################################
#
def genSamples(vehInOneLane,redVehs):
    
    samplesAll = []#收集当前车道内对应红车所有样本


    for iRed, redID in enumerate(redVehs.vehicle_id.unique()):
        print(iRed,redID)
        
        redVehFocusTmp = redVehs[redVehs.vehicle_id == redID]#红灯状态的车辆ID
        timeList = redVehFocusTmp.timestep_time.values  

        vehInOneLane =vehInOneLane.sort_values(by='timestep_time',ascending=True)#提取持续的时间段
        maxLanePos =  max(vehInOneLane.vehicle_pos)#车道的长度
        
        
        
        for t in timeList:#枚举红灯状态下的每个时间的每一辆车 
            
      
            locTmp1 =  vehInOneLane.timestep_time == t      
            locTmp2 = abs(maxLanePos - vehInOneLane.vehicle_pos)<200 #距离红灯200米以内
            vehsAtTime = vehInOneLane[locTmp1 & locTmp2] #符合车道，时间和距离限制
                
                
            if len(vehsAtTime.vehicle_id.unique()) == 1:  # 如果当前时间车辆只有一部车，统计忽略
                #print("time",t,"len(vehsAtTime.vehicle_id.unique()) == 1")
                continue

           
            vehsAtTime =vehsAtTime.sort_values(by='vehicle_pos',ascending=False)
            counter = 0

            # 枚举当前道路上红灯状态下的每个时间的每一辆车，并生成每个时刻样本
            recordPerVeh = [];#记录每一辆车的[位置和速度]

            for rowindex, veh in vehsAtTime.iterrows():#每一辆车                  
                vehX =  veh.vehicle_x
                vehY =  veh.vehicle_y
                vehVel = veh.vehicle_speed
                vehTime  = veh.timestep_time
                vehID = veh.vehicle_id
                vehicle_Red_distane = maxLanePos - veh.vehicle_pos
                #print("vehID:",vehID," vehicle_Red_distane:",vehicle_Red_distane)

                if counter == 0 and vehID != redID:
                    #第一部车不是红灯静止车，有错误
                    print("counter == 0 and vehID != redID:",vehID,redID)
                    break

                recordPerVeh.extend([vehicle_Red_distane, vehVel])#记录每一辆车的[位置和速度]

                if counter > 0 and counter <19:#生成用于机器学习的样本,车辆不超过20
                    avg_speed_lane = 60/3.6
                    max_speed_lane = 80/3.6

                    #subject为主车样本
                    subject = [vehID,max(timeList) - t,#主车名，红灯剩余时间
                    vehicle_Red_distane,
                    vehVel,
                    avg_speed_lane,
                    vehicle_Red_distane/(vehVel+0.01),
                    vehicle_Red_distane/avg_speed_lane]


                    #samplesTmp1为当前时刻主车前面的车（最大20车）车辆的状态
                    samplesTmp1 = deepcopy(recordPerVeh)
                    samplesTmp1.extend([0, 0]*(19-counter))#主车前面的车（最大20车）车辆的状态


                    #samplesTmp2为当前时刻当前样本：主车+主车前面的车（最大20车）+speedFlag
                    samplesTmp2 = deepcopy(subject)#当前样本：主车
                    samplesTmp2.extend(samplesTmp1)#当前样本：主车+主车前面的车（最大20车）
                    samplesTmp2.extend([speedFlagDict[vehID]])


                    samplesAll.append(samplesTmp2)#收集所有的样本
                    
                counter = counter+1


            
            
   
    return samplesAll

###################################################################################################
#根据交通灯边红色静止车的特征，获得符合特征的红灯静止车

def extractRedVehs(vehInOneLane):
    
    redVehs = pd.DataFrame(columns=vehInOneLane.columns)  #建立空的二维数组，并且列与数据库一致。
    
    maxLanePos =  max(vehInOneLane.vehicle_pos)#车道的长度
    
    ####提取红灯车辆以及时刻
    vehInOneLane =vehInOneLane.sort_values(by='timestep_time',ascending=True)#提取持续的时间段
    timeList = vehInOneLane.timestep_time.unique()

    lowSpeedFlag = 0
    for t in timeList:#枚举每个时间，进行分析，获得红灯车辆
        
        vehsAtTime = vehInOneLane[vehInOneLane.timestep_time == float(t)]#当前时刻，当前车道上的所有车

        for index, veh in vehsAtTime.iterrows():#每一辆车
            #print(index,veh)
            vehX =  veh.vehicle_x
            vehY =  veh.vehicle_y
            vehVel = veh.vehicle_speed
            vehTime  = veh.timestep_time
            vehID = veh.vehicle_id
            vehicle_pos = veh.vehicle_pos
            if (vehVel <2/3.6) and (abs(vehicle_pos - maxLanePos)<3):#红色静止车辨别标准
                lowSpeedFlag = 1
                redVehs.loc[len(redVehs.index)] =veh
                
    return redVehs
    
###################################################################################################
#根据交通灯边红色静止车的特征，获得符合特征的所有其他车的最小速度
#认为其他车的最小速度是从计时开始到离开当前车道

def analyzingRedVehAtCurLane(redVehs,vehInOneLane,curLaneID):
    
    speedFlagDict = dict()
    maxLanePos =  max(vehInOneLane.vehicle_pos)#车道的长度
    
    for iRed, redID in enumerate(redVehs.vehicle_id.unique()):

        redVehFocusTmp = redVehs[redVehs.vehicle_id == redID]#红灯状态的车辆ID
        timeList = redVehFocusTmp.timestep_time.values  #红灯车的持续时间

        locTmp1 = vehInOneLane.timestep_time >= min(timeList)
        locTmp2 = vehInOneLane.timestep_time <= max(timeList)
        locTmp3 = abs(maxLanePos - vehInOneLane.vehicle_pos)<200 ##距离红灯200米以内,红灯时间内车道内所有的车

        #红灯时间内车道内所有的车,而且必须距离红灯200米以内，排除绿灯时在道路的车,核心数据1
        vehsAtTimeAndDist = vehInOneLane[locTmp1 & locTmp2 & locTmp3]
        vehIDsAtTimeAndDist = vehsAtTimeAndDist.vehicle_id.unique()#符合条件的所有车
        #print(vehsAtTimeAndDist.head(5))
        
        for ii,idTmp  in enumerate(vehIDsAtTimeAndDist):

            #提取符合ID的车，注意采用的是vehInOneLane，不是vehsAtTimeAndDist
            vehTmp =  vehInOneLane[vehInOneLane.vehicle_id==idTmp]

            #提取符合时间的车，  locTmp1 = vehInOneLane.timestep_time >= min(timeList)
            vehTmp = vehTmp[vehTmp.timestep_time >= min(timeList)]

            #提取符合位置的的车
            locTmp3 = abs(maxLanePos - vehTmp.vehicle_pos)<200
            vehTmp = vehTmp[locTmp3]

            minSpeed = min(vehTmp.vehicle_speed.values)#距离红灯100米以内,红灯时间内一部车的是所有速度信息
            if minSpeed >= 35/3.6:
                speedFlag  = 4
            if minSpeed <=35/3.6 and minSpeed> 25/3.6:
                speedFlag  = 3
            if minSpeed <=25/3.6 and minSpeed> 15/3.6:
                speedFlag  = 2
            if minSpeed <=15/3.6 and minSpeed> 5/3.6:
                speedFlag  = 1
            if minSpeed <=5/3.6:
                speedFlag  = 0
            #注意时间分区
            speedFlagDict[idTmp] = speedFlag


    return speedFlagDict



###################################################################################################
#############主程序

df = pd.read_csv('./trainData/originFranceData1.csv',sep = ';')

laneList = df.vehicle_lane.unique()#获得每一条道路
numLanes = len(df.vehicle_lane.unique())


########################################################################################
#######枚举每一个车道,获得红灯附近的车，以及车道的长度
for ilane,curLaneID in enumerate(df.vehicle_lane.unique()):#枚举每一个车道
  
    print("laneIndex is %d,nameID is %s" %(ilane,curLaneID))
    
    if ilane<125:
        continue
    
    curLaneID= laneList[ilane]

    redVehs = pd.DataFrame(columns=df.columns)  #建立空的二维数组，并且列与数据库一致。
    
    vehInOneLane = df[df.vehicle_lane==curLaneID]#获得当前车道上所有车辆
    
    if vehInOneLane.empty == True:#当前车道没有车
        continue
        
    ###提取交通灯边的静止车    
    redVehs = extractRedVehs(vehInOneLane) 
    if redVehs.empty == True:#如果当前车道没有红灯车
         continue

            
    ####给出每辆车的最小速度        
    speedFlagDict = analyzingRedVehAtCurLane(redVehs,vehInOneLane,curLaneID)
    

    #并生成样本\n")
    samplesAll=genSamples(vehInOneLane,redVehs) 
        
    print("len(speedFlagDict):",len(speedFlagDict))
    print("len(redVehs):",len(redVehs))
    print("len(samplesAll):",len(samplesAll))
        
    #当前车道，每个红灯车的所有时刻的样本
    name1 = ["vehID","redLightTime","distToRedLight","speed","laneAvgSpeed","arriveTime1","arriveTime2"]   
    name2 = ["vehPos_1","vehSpeed_1","vehPos_2","vehSpeed_2","vehPos_3","vehSpeed_3","vehPos_4","vehSpeed_4"] 
    name3 = ["vehPos_5","vehSpeed_5","vehPos_6","vehSpeed_6","vehPos_7","vehSpeed_7","vehPos_8","vehSpeed_8"]
    name4 = ["vehPos_9","vehSpeed_9","vehPos_10","vehSpeed_10","vehPos_11","vehSpeed_11","vehPos_12","vehSpeed_12"]
    name5 = ["vehPos_13","vehSpeed_13","vehPos_14","vehSpeed_14","vehPos_15","vehSpeed_15","vehPos_16","vehSpeed_16"]
    name6 = ["vehPos_17","vehSpeed_17","vehPos_18","vehSpeed_18","vehPos_19","vehSpeed_19","vehPos_20","vehSpeed_20"]
    headers = name1+name2+name3+name4+name5+name6+["speedFlag"]

    if samplesAll != []:
        print(samplesAll[0])
        samplesTmp = pd.DataFrame(samplesAll,columns=headers)
        print(samplesTmp.info())
        filename = './franceRedData/'+str(ilane)+'+'+curLaneID+'.csv'
        samplesTmp.to_csv(filename,float_format='%.3f',index=0) 


###################################################################################
#将所有样本集合成一个CSV文件
import os
import pandas as pd
path = "./franceRedData/"

filelist = [path + i for i in os.listdir(path)]
dataset = pd.read_csv(filelist[0])

for tmpFile in filelist:
    if tmpFile.endswith(".csv"):
        #print(tmpFile)
        tmpDF = pd.read_csv(tmpFile)
        dataset = pd.concat([dataset,tmpDF],ignore_index=True,axis=0)
        

filename= "./trainData/"+"0_allSamples.csv" 
dataset.to_csv(filename,float_format='%.3f',index=0) 
        



laneIndex is 0,nameID is -1356_0
laneIndex is 1,nameID is 28925528_0
laneIndex is 2,nameID is 239331354_0
laneIndex is 3,nameID is -240120122_0
laneIndex is 4,nameID is -1274_0
laneIndex is 5,nameID is -1354_0
laneIndex is 6,nameID is -1280#0_0
laneIndex is 7,nameID is -1328#0_0
laneIndex is 8,nameID is 229259796#0_0
laneIndex is 9,nameID is -1324_0
laneIndex is 10,nameID is 28925528_1
laneIndex is 11,nameID is -1328#0_1
laneIndex is 12,nameID is :431399507_0_0
laneIndex is 13,nameID is :-584_1_0
laneIndex is 14,nameID is 229259796#0_1
laneIndex is 15,nameID is :-596_7_0
laneIndex is 16,nameID is -1324_1
laneIndex is 17,nameID is -1354_1
laneIndex is 18,nameID is -1354_2
laneIndex is 19,nameID is 229259796#0_2
laneIndex is 20,nameID is -1328#1_0
laneIndex is 21,nameID is :-228_1_0
laneIndex is 22,nameID is -1280#2_0
laneIndex is 23,nameID is -1336#0_0
laneIndex is 24,nameID is --1288_0
laneIndex is 25,nameID is 239331354_1
laneIndex is 26,nameID is -1280#1_0
laneIndex is 27,nameID is -

120 VehicleFlowWestToEast1.36
121 VehicleFlowWestToEast1.38
122 VehicleFlowWestToEast1_0.44
123 VehicleFlowWestToEast1_0.38
124 VehicleFlowWestToEast1.40
125 VehicleFlowSouthToEast1.13
126 VehicleFlowWestToEast1.43
127 VehicleFlowWestToEast1_0.49
128 VehicleFlowNorthToEast1.10
129 VehicleFlowWestToEast1.44
130 VehicleFlowNorthToEast1.11
131 VehicleFlowWestToEast1_0.51
132 VehicleFlowWestToEast1.47
133 VehicleFlowWestToEast1.51
134 VehicleFlowWestToEast1.54
135 VehicleFlowWestToEast1_0.55
136 VehicleFlowWestToEast1.53
137 VehicleFlowWestToEast1_0.59
138 VehicleFlowSouthToEast1.16
139 VehicleFlowWestToEast1.59
140 VehicleFlowWestToEast1_0.61
141 VehicleFlowWestToEast1.57
142 VehicleFlowNorthToEast1.13
143 VehicleFlowWestToEast1.65
144 VehicleFlowWestToEast1_0.69
145 VehicleFlowNorthToEast1.14
146 VehicleFlowWestToEast1.66
147 VehicleFlowWestToEast1_0.72
148 VehicleFlowWestToEast1.68
149 VehicleFlowWestToEast1_0.75
150 VehicleFlowWestToEast1.73
151 VehicleFlowWestToEast1.71
152 VehicleFlo

389 VehicleFlowWestToEast4.83
390 VehicleFlowWestToEast6_0.4
391 VehicleFlowWestToEast6_0.7
392 VehicleFlowNorthToEast6.3
393 VehicleFlowWestToEast6_0.6
394 VehicleFlowNorthToEast6.4
395 VehicleFlowWestToEast6_0.10
396 VehicleFlowWestToEast6.11
397 VehicleFlowWestToEast6.12
398 VehicleFlowWestToEast6_0.8
399 VehicleFlowNorthToEast6.5
400 VehicleFlowWestToEast6.6
401 VehicleFlowWestToEast6.14
402 VehicleFlowSouthToEast6.0
403 VehicleFlowWestToEast6.8
404 VehicleFlowWestToEast6_0.17
405 VehicleFlowNorthToEast6.6
406 VehicleFlowWestToEast6_0.18
407 VehicleFlowSouthToEast6.2
408 VehicleFlowWestToEast6.17
409 VehicleFlowWestToEast6_0.19
410 VehicleFlowWestToEast6.18
411 VehicleFlowWestToEast6.19
412 VehicleFlowSouthToEast6.4
413 VehicleFlowWestToEast6_0.26
414 VehicleFlowWestToEast6_0.31
415 VehicleFlowWestToEast6.27
416 VehicleFlowWestToEast6_0.33
417 VehicleFlowWestToEast6.28
418 VehicleFlowWestToEast6_0.38
419 VehicleFlowWestToEast6_0.35
420 VehicleFlowWestToEast6.34
421 VehicleFlowWestT

laneIndex is 126,nameID is :1214943548_3_2
0 VehicleFlowWestToNorth5.35
1 VehicleFlowWestToEast5_0.88
len(speedFlagDict): 3
len(redVehs): 3
len(samplesAll): 1
['VehicleFlowWestToEast5.82', 0.0, 7.71, 4.59, 16.666666666666668, 1.6760869565217393, 0.46259999999999996, 1.7199999999999998, 0.29, 7.71, 4.59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 48 columns):
vehID             1 non-null object
redLightTime      1 non-null float64
distToRedLight    1 non-null float64
speed             1 non-null float64
laneAvgSpeed      1 non-null float64
arriveTime1       1 non-null float64
arriveTime2       1 non-null float64
vehPos_1          1 non-null float64
vehSpeed_1        1 non-null float64
vehPos_2          1 non-null float64
vehSpeed_2        1 non-null float64
vehPos_3          1 non-null int64
vehSpeed_3        1 non-null int64
vehPos_

5 VehicleFlowEastToSouth.4
6 VehicleFlowNorthToEast.13
7 VehicleFlowNorthToSouth.13
8 VehicleFlowNorthToEast.20
9 VehicleFlowNorthToEast2.2
10 VehicleFlowNorthToEast2.10
11 VehicleFlowNorthToEast2.17
12 VehicleFlowEastToSouth2.6
13 VehicleFlowNorthToEast3.1
14 VehicleFlowEastToSouth3.5
15 VehicleFlowNorthToSouth4.8
16 VehicleFlowNorthToEast6.2
17 VehicleFlowNorthToEast6.3
18 VehicleFlowNorthToEast6.9
19 VehicleFlowNorthToEast5.2
20 VehicleFlowNorthToEast7.0
21 VehicleFlowNorthToSouth7.0
len(speedFlagDict): 24
len(redVehs): 204
len(samplesAll): 12
['VehicleFlowNorthToEast.1', 9.0, 6.37, 0.42, 16.666666666666668, 14.813953488372093, 0.3822, 0.6100000000000003, 0.0, 6.37, 0.42, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 48 columns):
vehID             12 non-null object
redLightTime      12 non-null float64
distToRedLight    12 non-nu

103 VehicleFlowWestToEast4.67
104 VehicleFlowWestToEast4_0.75
105 VehicleFlowSouthToEast4.29
106 VehicleFlowWestToEast4_0.82
107 VehicleFlowSouthToEast4.35
108 VehicleFlowWestToEast4.85
109 VehicleFlowWestToEast6.3
110 VehicleFlowSouthToEast4.40
111 VehicleFlowWestToEast6.15
112 VehicleFlowWestToEast6.18
113 VehicleFlowSouthToEast6.5
114 VehicleFlowWestToEast6.29
115 VehicleFlowWestToEast6.34
116 VehicleFlowWestToEast6_0.42
117 VehicleFlowWestToEast6.42
118 VehicleFlowWestToEast6_0.49
119 VehicleFlowWestToEast6_0.56
120 VehicleFlowWestToEast6.58
121 VehicleFlowSouthToEast6.18
122 VehicleFlowWestToEast6.65
123 VehicleFlowWestToEast6_0.73
124 VehicleFlowWestToEast6.74
125 VehicleFlowWestToEast6_0.81
126 VehicleFlowWestToEast5_0.0
127 VehicleFlowNorthToEast5.0
128 VehicleFlowWestToEast5_0.7
129 VehicleFlowSouthToEast5.2
130 VehicleFlowWestToEast5.16
131 VehicleFlowWestToEast5.20
132 VehicleFlowSouthToEast5.10
133 VehicleFlowWestToEast5.25
134 VehicleFlowNorthToEast5.9
135 VehicleFlowSouth

laneIndex is 141,nameID is -1346_0
laneIndex is 142,nameID is -1340#0_0
laneIndex is 143,nameID is --1342_0
laneIndex is 144,nameID is -1326#0_0
laneIndex is 145,nameID is :-564_0_1
laneIndex is 146,nameID is -1326#0_1
laneIndex is 147,nameID is 105490271#1_1
laneIndex is 148,nameID is :2479049221_8_0
0 VehicleFlowWestToNorth5.51
len(speedFlagDict): 1
len(redVehs): 1
len(samplesAll): 0
laneIndex is 149,nameID is :2479049221_8_1
laneIndex is 150,nameID is -1326#1_0
laneIndex is 151,nameID is -1326#1_1
laneIndex is 152,nameID is :-702_1_0
laneIndex is 153,nameID is --1336#0_0
laneIndex is 154,nameID is -1326#2_0
laneIndex is 155,nameID is :1214943442_1_0
0 BusFlowNorthSouth3.1
1 VehicleFlowNorthToSouth6.2
2 VehicleFlowNorthToSouth6.7
len(speedFlagDict): 3
len(redVehs): 57
len(samplesAll): 0
laneIndex is 156,nameID is :1214943442_1_1
laneIndex is 157,nameID is -1326#2_1
laneIndex is 158,nameID is :1214943442_1_4
laneIndex is 159,nameID is :-808_0_0
laneIndex is 160,nameID is :431399476_0_